In [ ]:
!pip install openai
!pip install sklearn

In [ ]:
# 미국 뉴스 데이터에서 야구, 하키 뉴스 가져오기
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai
import os
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
categories = ['rec.sport.baseball', 'rec.sport.hockey']

# subset : 데이터 훈련용/테스트용/전체 선택 가능
# suffle : 데이터를 로드할때 섞을지 여부 지정
# random_state : 데이터 셔플링 시 랜덤 시드 설정
# categories : 로드할 뉴스 그룹 카테고리 지정
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

# 다운로드 받은 데이터 일부 확인
print(sports_dataset['data'][0])

# Data Exploration
- 데이터 탐색

In [ ]:
sports_dataset.keys()
# sports_dataset
sports_dataset.target_names[sports_dataset['target'][0]]

In [ ]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

# Data Preparation
- 데이터 전처리 : 불필요한 데이터 제거 & 텍스트만 추출
- Pandas를 이용해 prompt와 completion으로 정제함

In [ ]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']] # rec.sport.baseball --> baseball
texts = [text.strip() for text in sports_dataset['data']]                                   #data 중, text만 추려냄.
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()

In [101]:
df.to_json("sport2.jsonl", orient='records', lines=True)

# Data Preparation tool
- 텍스트 데이터를 GPT 모델을 위한 파인 튜닝용 형식으로 준비하는데 사용

In [ ]:
# 이 함수는 데이터를 특정 형식으로 변환하고, 모델이 학습할 수 있도록 전처리
# OpenAI에서 제공하는 tool의 기능으로, fine_tuning_exam.ipynb에서 수행했던 싱글쿼터('), 불필요한 문자열 제거를 자동 수행해준다.
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

In [ ]:
!pip install token-count
!pip install jsonlines

In [104]:
# 토큰 카운트
# 주어진 json lines 파일에서 특정 수의 토큰 이상을 포함하지 않도록 데이터를 조정한후
# 조정한 데이터를 새로운 파일에 저장
# 이 함수는 텍스트 데이터를 특정 토큰 제한 내에서 처리하는데 유용
# 파일의 'prompt'와 'completetion' 필드의 토큰 수를 계산하고, 특정 토큰 수를 초과하지 않도록 조정. 조정후 새 파일에 저장
from token_count import TokenCount
import json
import jsonlines

def token_limit(file_name, update_file_name, token_cnt):
  tc = TokenCount(model_name='gpt-3.5-turbo')
  #print(tc.num_tokens_from_file(file_name))

  tot_tokens=0
  update_txt =[]
  tmp1 =""
  tmp_cnt= 0

  with jsonlines.open(file_name) as f:
    for line in f.iter():
      tmp1 = ""
      tmp_cnt = 0
      for token in line["prompt"].split(' '):
        tmp1 += token
        tmp_cnt += 1

        if tmp_cnt > 500:
          break
      line["prompt"]=tmp1
      #print(line["prompt"])

      tokens = tc.num_tokens_from_string(line["prompt"])
      tokens2 = tc.num_tokens_from_string(line["completion"])
      tot_tokens += tokens + tokens2
      if tot_tokens > token_cnt:
        break
 #     print(tot_tokens)
      update_txt.append(line)

#  print(tot_tokens)
#  print(update_txt)

  with open(update_file_name, 'w', encoding="utf-8") as f:
    for item in update_txt:
      json.dump(item, f, ensure_ascii=False)
      f.write("\n")


# Fine-tuning

In [105]:
#파일 업데이트  함수 추가
import json

def update_data(file, update_file):
  data_list = []
# 기존 프롬프트-완료 데이터 로드
  data_list = []
  with open(file, "r") as f:
      for line in f:
          data_list.append( json.loads(line) )

  # 채팅 형식으로 변환
  chat_data = []
  for entry in data_list:
      chat_entry = {
          "messages": [
              {"role": "system", "content": "당신은 친절한 비서입니다."},  # 시스템 메시지는 필요에 따라 변경 가능
              {"role": "user", "content": entry["prompt"]},
              {"role": "assistant", "content": entry["completion"].strip()}
          ]
      }
      chat_data.append(chat_entry)

  str=""
  for item in chat_data:
    str += json.dumps(item) + "\n"

  # 변환된 데이터를 새 파일로 저장
  with open(update_file, "w", encoding="utf-8") as f:
    for i in chat_data:
      json.dump(i, f, ensure_ascii=False) # ensure_ascii로 한글이 깨지지 않게 저장
      f.write("\n") # json을 쓰는 것과 같지만, 여러 줄을 써주는 것이므로 "\n"을 붙여준다.

In [106]:
token_limit("sport2_prepared_train.jsonl", "sport2_prepared_train_drop.jsonl", 4000)
token_limit("sport2_prepared_valid.jsonl", "sport2_prepared_valid_drop.jsonl", 4000)

update_data("sport2_prepared_train_drop.jsonl", "sport2_prepared_train_drop.jsonl")                   # 업데이트 함수 추가
update_data("sport2_prepared_valid_drop.jsonl", "sport2_prepared_valid_drop.jsonl")                   # 업데이트 함수 추가

train_file = client.files.create(file=open("sport2_prepared_train_drop.jsonl", "rb"), purpose="fine-tune") # 트레이닝 파일
valid_file = client.files.create(file=open("sport2_prepared_valid_drop.jsonl", "rb"), purpose="fine-tune") # 결과물을 체크하는 파일

# 시간이 꽤 걸린다
fine_tuning_job = client.fine_tuning.jobs.create(training_file=train_file.id, validation_file=valid_file.id, model="gpt-4o-2024-08-06")

In [ ]:
print(fine_tuning_job)

In [ ]:
# 특정 파인 튜닝 작업이 세부 정보를 가져옴
fine_tune_results = client.fine_tuning.jobs.retrieve(fine_tuning_job.id).result_files
result_file = client.files.retrieve(fine_tune_results[0])
content = client.files.content(result_file.id)

with open("result.csv", "wb") as f:
    f.write(content.text.encode("utf-8"))

In [115]:
# 위의 result_file에 저장된 값을 테이블 형식으로 변경해서 저장 - 사람이 볼 수 있도록 인코딩
import base64
base64.b64decode(content.text.encode("utf-8"))

# To save contet to file
with open("result.csv", "wb") as f:
  f.write(base64.b64decode(content.text.encode("utf-8")))

In [116]:
results = pd.read_csv('result.csv')
results[results['train_accuracy'].notnull()].tail(1)
print(results)

    step  train_loss  train_accuracy  valid_loss  valid_mean_token_accuracy  \
0      1     7.30027            0.25     9.26150                       0.25   
1      2    10.73621            0.25     9.44178                       0.50   
2      3     9.56514            0.25    10.81888                       0.25   
3      4     8.66793            0.25     8.52589                       0.25   
4      5     7.14792            0.25     7.48306                       0.25   
..   ...         ...             ...         ...                        ...   
94    95     0.00104            1.00     0.00040                       1.00   
95    96     0.00010            1.00     0.00039                       1.00   
96    97     0.00031            1.00     0.00021                       1.00   
97    98     0.00018            1.00     0.00011                       1.00   
98    99     0.00066            1.00     0.00011                       1.00   

    train_mean_reward  full_validation_mean_reward 

In [ ]:
!pip install matplotlib

In [ ]:
# 머신러닝 및 딥러닝 모델을 훈련할 때, 훈련 데이터셋이 모델의 예측 정확도를 나타내는 지표
# 얼마나 잘 맞는지 평가하는데 사용
results[results['train_accuracy'].notnull()]['train_accuracy'].plot()

# 모델 사용
- 파인 튜닝 된 모델을 이용해 분류를 잘 하는지 확인 (baseball, hockey)

In [146]:
test = pd.read_json('sport2_prepared_valid_drop.jsonl', lines=True)
# test = pd.read_json('sport2_prepared_valid.jsonl', lines=True)
test.head()

,messages
0,"[{'role': 'system', 'content': '당신은 친절한 비서입니다...."
1,"[{'role': 'system', 'content': '당신은 친절한 비서입니다...."
2,"[{'role': 'system', 'content': '당신은 친절한 비서입니다...."
3,"[{'role': 'system', 'content': '당신은 친절한 비서입니다...."
4,"[{'role': 'system', 'content': '당신은 친절한 비서입니다...."


In [152]:
ft_model = client.fine_tuning.jobs.retrieve('ftjob-IVfGEWOsM47hBGCcGQYcm5Cg').fine_tuned_model

# gpt-3.5 방식
# res = client.completions.create(model=ft_model, prompt=test['prompt'][5] + '\n\n###\n\n', max_tokens=1, temperature=0)
# gpt-4o 방식
messages = test.loc[0, 'messages'] # 첫 번째 행의 'messages' 필드 가져오기
user_message = next((msg['content'] for msg in messages if msg['role'] == 'user'), "")
res = client.chat.completions.create(
    model=ft_model, 
     messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_message + '\n\n###\n\n'}
    ], 
    max_tokens=10, 
    temperature=0)

# res.choices[0].text # gpt-3.5 방식
response_text = res.choices[0].message.content
print(response_text)

# OpenAI API를 통해 텍스트를 완성하는데 사용하는 함수
# model:사용할 모델 지정, prompt: 프롬프트 지정, max_tokens: 생성할 최대 토큰 수, temperature: 생성 텍스트의 창의성 조정, n: 생성할 완성의 수, stop: 텍스트 생성을 멈출 문자열, logprobs: 모델이 예측한 각 토큰의 로그 확률을 반환

hockey


# 범용화
- 모델에 다른 프롬프트를 입력해서 동작하는지 확인

In [159]:
# teweet에 있는 내용으로 테스트
sample_hockey_tweet = """Thank you to the
@Canes
 and all you amazing Caniacs that have been so supportive! You guys are some of the best fans in the NHL without a doubt! Really excited to start this new chapter in my career with the
@DetroitRedWings
 !!"""
# res = client.completions.create(model=ft_model, prompt=sample_hockey_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
# res.choices[0].text

# GPT-4 채팅 모델을 사용하여 요청
res = client.chat.completions.create(
    model=ft_model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": sample_hockey_tweet + '\n\n###\n\n'}
    ],
    max_tokens=10,  # 토큰 수를 적절하게 설정
    temperature=0
)

# 응답 내용 확인
response_text = res.choices[0].message.content
print(response_text)


hockey


In [160]:
sample_baseball_tweet="""BREAKING: The Tampa Bay Rays are finalizing a deal to acquire slugger Nelson Cruz from the Minnesota Twins, sources tell ESPN."""
# res = client.completions.create(model=ft_model, prompt=sample_baseball_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
# res.choices[0].text

# GPT-4 채팅 모델을 사용하여 요청
res = client.chat.completions.create(
    model=ft_model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": sample_baseball_tweet + '\n\n###\n\n'}
    ],
    max_tokens=10,  # 토큰 수를 적절하게 설정
    temperature=0
)

# 응답 내용 확인
response_text = res.choices[0].message.content
print(response_text)



baseball
